# Text-to-SQL Demo with Medha and SQLite

This notebook walks through a complete Text-to-SQL workflow:

1. Create an in-memory SQLite database with sample data
2. Store question-SQL pairs in Medha's semantic cache
3. Demonstrate all four cache tiers:
   - **Tier 0** — L1 in-memory cache (microseconds)
   - **Tier 1** — Template matching with parameter extraction
   - **Tier 2** — Exact vector match
   - **Tier 3** — Semantic similarity match

**Requirements:** `pip install medha[fastembed]`

## Cell 1: Setup & Imports

In [ ]:
import sqlite3
import time

from medha import Medha, Settings
from medha.embeddings.fastembed_adapter import FastEmbedAdapter
from medha.types import QueryTemplate

## Cell 2: Create a SQLite Database

We create an in-memory SQLite database with two tables:
- `employees` — 5 rows with name, department, and salary
- `products` — 3 rows with name, category, and price

In [ ]:
conn = sqlite3.connect(":memory:")
conn.executescript("""
    CREATE TABLE employees (id INT, name TEXT, department TEXT, salary REAL);
    INSERT INTO employees VALUES (1, 'Alice', 'Engineering', 95000);
    INSERT INTO employees VALUES (2, 'Bob', 'Marketing', 72000);
    INSERT INTO employees VALUES (3, 'Charlie', 'Engineering', 105000);
    INSERT INTO employees VALUES (4, 'Diana', 'Sales', 68000);
    INSERT INTO employees VALUES (5, 'Eve', 'Engineering', 112000);

    CREATE TABLE products (id INT, name TEXT, category TEXT, price REAL);
    INSERT INTO products VALUES (1, 'Widget A', 'Gadgets', 29.99);
    INSERT INTO products VALUES (2, 'Widget B', 'Gadgets', 49.99);
    INSERT INTO products VALUES (3, 'Gizmo X', 'Tools', 89.99);
""")
print("SQLite database created with 'employees' and 'products' tables.")

## Cell 3: Initialize Medha

We use:
- **FastEmbedAdapter** — local embeddings, no API key needed
- **memory mode** — Qdrant runs in-process, no external server required
- **query_language="sql"** — labels the cache as SQL-oriented

In [ ]:
embedder = FastEmbedAdapter()
settings = Settings(qdrant_mode="memory", query_language="sql")

medha = Medha(collection_name="sql_demo", embedder=embedder, settings=settings)
await medha.start()
print(f"Medha initialized (collection='sql_demo', mode='memory', language='sql')")

## Cell 4: Store Question-SQL Pairs

We store 6 question-SQL pairs. Each SQL query is executed against the real SQLite database
and the result is stored as `response_summary` — so the cache contains both the query and its output.

In [ ]:
pairs = [
    ("How many employees are there?", "SELECT COUNT(*) FROM employees"),
    ("List all employees in Engineering", "SELECT * FROM employees WHERE department = 'Engineering'"),
    ("Show me the top earners", "SELECT * FROM employees ORDER BY salary DESC LIMIT 5"),
    ("What is the average salary?", "SELECT AVG(salary) FROM employees"),
    ("Get all products", "SELECT * FROM products"),
    ("How many products cost more than 50?", "SELECT COUNT(*) FROM products WHERE price > 50"),
]

for question, sql in pairs:
    result = conn.execute(sql).fetchall()
    await medha.store(question, sql, response_summary=str(result))
    print(f"Stored: {question!r}  ->  {sql}  ->  {result}")

## Cell 5: Tier 2 — Exact Vector Match

When we search with the **exact same question** that was stored, Medha finds an exact
vector match (cosine similarity ~0.99+). This is Tier 2 in the waterfall.

In [ ]:
start = time.perf_counter()
hit = await medha.search("How many employees are there?")
elapsed = (time.perf_counter() - start) * 1000

print(f"Strategy: {hit.strategy}")        # EXACT_MATCH or L1_CACHE
print(f"Query:    {hit.generated_query}")  # SELECT COUNT(*) FROM employees
print(f"Score:    {hit.confidence:.4f}")   # ~0.99+
print(f"Time:     {elapsed:.2f}ms")

## Cell 6: Tier 3 — Semantic Similarity Match

Now we search with a **rephrased question** that was never stored. Medha recognizes the
semantic similarity and returns the matching SQL query from the cache.

In [ ]:
start = time.perf_counter()
hit = await medha.search("What's the total number of staff members?")
elapsed = (time.perf_counter() - start) * 1000

print(f"Strategy: {hit.strategy}")        # SEMANTIC_MATCH
print(f"Query:    {hit.generated_query}")  # SELECT COUNT(*) FROM employees
print(f"Score:    {hit.confidence:.4f}")   # ~0.90+
print(f"Time:     {elapsed:.2f}ms")

## Cell 7: Tier 1 — Template Matching with Parameter Extraction

Templates define parameterized patterns. When a question matches a template,
Medha extracts parameters via regex and fills them into the query template.

Here we load a template for filtering employees by department and test it
with "Marketing" — a parameter value that was **not** in any stored pair.

In [ ]:
templates = [
    QueryTemplate(
        intent="employees_by_dept",
        template_text="List employees in {department}",
        query_template="SELECT * FROM employees WHERE department = '{department}'",
        parameters=["department"],
        parameter_patterns={"department": r"\b(Engineering|Marketing|Sales|HR)\b"},
    ),
]

await medha.load_templates(templates)
print(f"Loaded {len(templates)} template(s)\n")

start = time.perf_counter()
hit = await medha.search("Show me employees in Marketing")
elapsed = (time.perf_counter() - start) * 1000

print(f"Strategy: {hit.strategy}")        # TEMPLATE_MATCH
print(f"Query:    {hit.generated_query}")  # SELECT * FROM employees WHERE department = 'Marketing'
print(f"Time:     {elapsed:.2f}ms\n")

# Execute the generated query against SQLite to verify it works
result = conn.execute(hit.generated_query).fetchall()
print(f"Results:  {result}")

## Cell 8: Tier 0 — L1 In-Memory Cache

The L1 cache stores recent search results in memory. The first call goes through the
full waterfall (vector search, scoring, etc.). The second call for the **same question**
returns instantly from the L1 cache — typically >100x faster.

In [ ]:
# First call — goes through the vector backend
start = time.perf_counter()
hit1 = await medha.search("What is the average salary?")
t1 = (time.perf_counter() - start) * 1000

# Second call — served from L1 cache
start = time.perf_counter()
hit2 = await medha.search("What is the average salary?")
t2 = (time.perf_counter() - start) * 1000

print(f"First call:  {t1:.2f}ms ({hit1.strategy})")
print(f"Second call: {t2:.2f}ms ({hit2.strategy})")
if t2 > 0:
    print(f"Speedup:     {t1/t2:.0f}x")

## Cell 9: Stats & Cleanup

In [ ]:
print("Cache Statistics:")
print(medha.stats)

await medha.close()
conn.close()
print("\nCleaned up: Medha closed, SQLite connection closed.")